In [1]:
#Se vincula una cuenta de google para exportar los resultados a drive.

# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvcc --version       

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
# Instalamos la libreria de TaxoNERD. Esta es una version custom que se encuentra en mi perfil de GitHub

!pip install git+https://github.com/ppertuzduran/taxonerd.git#egg=taxonerd

In [ ]:
#Instalamos la libreria para el uso de transformers, de Hugging Face.

!pip install transformers[sentencepiece]

In [ ]:
#Importamos el tokenizador

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

# device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
# model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl").to(device)
model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
3
nlp_loc = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

In [ ]:
!pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.3.0/en_ner_eco_biobert-1.0.0.tar.gz

In [6]:
!pip show en_ner_eco_biobert

Name: en-ner-eco-biobert
Version: 1.0.0
Summary: A model based on BioBERT for taxonomic NER in ecological documents.
Home-page: https://github.com/nleguillarme/taxonerd
Author: Laboratoire d'Ecologie Alpine
Author-email: nicolas.leguillarme@univ-grenoble-alpes.fr
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: spacy
Required-by: 


In [ ]:
!pip install taxonerd[cuda111]

In [8]:
#Cargamos el modelo para extraer nombres científicos
from taxonerd import TaxoNERD
ner = TaxoNERD(model="en_ner_eco_biobert", prefer_gpu=True, with_abbrev=True) # Add with_linking="gbif_backbone" or with_linking="taxref" to activate entity linking

In [ ]:
!pip install pdfplumber

In [10]:
import pdfplumber
import textract
import os
import re
import pandas as pd
from glob import glob
from tqdm.autonotebook import tqdm

In [11]:
# import tkinter
# from tkinter import filedialog

# tkinter.Tk().withdraw() # prevents an empty tkinter window from appearing

# folder_path = filedialog.askdirectory()
folder_path = r"/content/drive/MyDrive/Tests/input"
output_path = r"/content/drive/MyDrive/Tests/output"

In [12]:
def extract_from_folder(input_path, output_path):
    #Si no existe una carpeta llamada "entities_from_docs" en la ruta
    #de salida, se crea.
    if not os.path.exists(output_path+"/entities_from_docs"):
        os.makedirs(output_path+"/entities_from_docs")
    
    IDs = []
    for filepath in glob(os.path.join(input_path, "*")): #se recorre cada documento
        basename = os.path.basename(filepath) #se guarda el nombre del documento
        IDs.append(basename) #se guardan los nombres de los documentos en la lista IDs
        print(f"\nProcessing file: {basename}")
        
        if basename.endswith(".pdf"): #si el documento es .pdf se procesa con pdfplumber
            text = process_pdf(filepath, basename)
            extract_from_text(text, basename) #Se extraen y exportan las entidades encontradas.

        else:
            text = textract.process(filepath).decode("utf-8") #si el documento no es .pdf se procesa con textract
            text = post_processing(text)
            extract_from_text(text, basename) #Se extraen y exportan las entidades encontradas.

    #se exporta la tabla IDs que contiene los documentos procesados    
    IDs = pd.DataFrame(IDs, columns=['ID']) 
    IDs.to_csv(output_path+"/"+"IDs.csv", header=False, index=False)

In [13]:
def process_pdf(filepath, basename):
    with pdfplumber.open(filepath) as pdf:
        # first_page = pdf.pages[0]
        pdf_lenght = len(pdf.pages)
        text = [pdf.pages[page].extract_text() 
                for page in range(pdf_lenght)]
        
        text = [text[chunk] 
                for chunk in range(pdf_lenght) 
                if text[chunk] is not None]
        
        text = "".join(text)
        text = post_processing(text)
    return text 

In [14]:
def extract_from_text(input_text, basename):    
    text_splited = get_split(input_text) #separa el texto con el objetivo de procesar conjuntos de max 512 tokens
    splited_text_lenght = len(text_splited)
    
    #TAXA

    #busca entidades taxonómicas en cada chunk
    taxa_entities = [ner.find_entities(text_splited[chunk]) 
                     for chunk in tqdm(range(splited_text_lenght))]
    
    #borra los dataframes vaciones generados (cuadno no hay entidades en un chunk)
    taxa_entities = [taxa_entities[df]['text'] 
                     for df in range(splited_text_lenght) 
                     if not taxa_entities[df].empty]

    #Si hay entidades en el documento:
    if taxa_entities:
        temp_tax = pd.concat(taxa_entities).reset_index(drop=True) #se unen los dataframes en uno solo
        # print(temp_tax)
        taxa_frame = pd.DataFrame() #se crea un nuevo dataframe con las entidades y su número de aparición
        taxa_frame['scientific_name'] = temp_tax.value_counts().index
        taxa_frame['ocurrence'] = list(temp_tax.value_counts())
        # print(taxa_frame)

        #Se convierte el dataframe con las entidades a un .csv y se exporta a la carpeta de salida
        # taxa_entities.to_csv(output_path+"/entities_from_docs/"+basename+"_taxa.csv", header=False, index=False) 
        taxa_frame.to_csv(output_path+"/entities_from_docs/"+basename+"_taxa.csv", index=False)    
        print(f"{basename}_taxa.csv exported")

    #LOCATIONS VERSION 2

    #Busca entidades referentes a localización en cada chunk
    loc_entities = [find_locs(text_splited[chunk]) 
                    for chunk in tqdm(range(splited_text_lenght))]

    #borra los dataframes vaciones generados (cuando no hay entidades en un chunk)
    loc_entities = [loc_entities[df]
                    for df in range(splited_text_lenght) 
                    if not loc_entities[df].empty]

    #Si hay entidades en el documento:
    if loc_entities:
        temp_loc = pd.concat(loc_entities).reset_index(drop=True) #se unen los dataframes en uno solo
        loc_frame = pd.DataFrame() #se crea un nuevo dataframe con las entidades y su número de aparición
        loc_frame['location'] = temp_loc.value_counts().index
        loc_frame['ocurrence'] = list(temp_loc.value_counts())
        # print(loc_frame)
    
        #Se convierte el dataframe con las entidades a un .csv y se exporta a la carpeta de salida
        loc_frame.to_csv(output_path+"/entities_from_docs/"+basename+"_locs.csv", index=False) 
        print(f"{basename}_locs.csv exported")

        # print("\nPRUEBA QUITAR LOCS EN TAXA DATAFRAME")
        
        # df1, df2 = taxa_frame, loc_frame2
        # cond = df1['scientific_name'].isin(df2['location'])
        # df1.drop(df1[cond].index, inplace = True)
        # print(df1)

In [15]:
def find_locs(text):
  entities = nlp_loc(text)
  df = pd.DataFrame(entities)
  if not df.empty:
    df = df.loc[df['entity_group'] == "LOC"]
    df = df.loc[df['score']>0.98]
    df = df['word']
  return df

In [16]:
def post_processing(text):
  #Replace \n by withespace
  text = re.sub("\n"," ", text)
  # Replace \xa0 by withespace
  text = re.sub("\xa0", " ", text)
  # Replace \t by whitespace
  text = re.sub("\t+", " ", text)
  # Remove word break
  text = re.sub("-\n", "", text)
  # Remove newline characters in paragraphs
  text = re.sub("(?<!\n)\n(?!\n)", " ", text)
  # Remove multiple whitespaces
  text = re.sub(" +", " ", text)
  # Remove trailing newlines
  text = text.strip(" \n")
  return text

In [17]:
def get_split(input_text):
  l_total = []
  l_parcial = []
  if len(input_text.split())//300 >0:
    n = len(input_text.split())//400
  else: 
    n = 1
  
  for w in range(n):
    if w == 0:
      l_parcial = input_text.split()[:400]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = input_text.split()[w*300:w*300 + 400]
      l_total.append(" ".join(l_parcial))
  return l_total

In [18]:
extract_from_folder(folder_path, output_path)


Processing file: CD-337.pdf


  0%|          | 0/20 [00:00<?, ?it/s]

CD-337.pdf_taxa.csv exported


  0%|          | 0/20 [00:00<?, ?it/s]

CD-337.pdf_locs.csv exported

Processing file: AjustesPlanillaDeEstructura.docx


...

PRY_BEM_ANH_012_15_ITF.pdf_locs.csv exported

Processing file: Lista de Bibliografia SIGMA.xlsx


  0%|          | 0/51 [00:00<?, ?it/s]

Lista de Bibliografia SIGMA.xlsx_taxa.csv exported


  0%|          | 0/51 [00:00<?, ?it/s]

Lista de Bibliografia SIGMA.xlsx_locs.csv exported
